In [1]:
import pymongo
import json
import pprint

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["yelpdb"]

In [3]:
mydb.collection_names()

['photo',
 'tip_aggregate_by_business',
 'user',
 'business_name',
 'business',
 'review',
 'tip',
 'checkin']

Perform MongoDB query within one collection

In [4]:
# Top 10 Chinese / Japanese restaurants in Las Vegas with count of review > 500
myquery = {
    "categories": {"$regex": ".*Chinese.*|.*Japanese.*" },
    "city": "Las Vegas",
    "review_count": {"$gt": 500},
    "is_open":1
}

query_result = mydb['business'].find(myquery, {'name':1}).sort("stars",-1).limit(10)
for res in query_result:
    pprint.pprint(res)

{'_id': ObjectId('5c719e093d10166478cba143'), 'name': 'Pan Asian'}
{'_id': ObjectId('5c719e093d10166478cbb798'), 'name': 'Island Style Restaurant'}
{'_id': ObjectId('5c719e093d10166478cc0168'),
 'name': 'Kaizen Fusion Roll and Sushi'}
{'_id': ObjectId('5c719e0a3d10166478cc2070'), 'name': 'Cafe Sanuki'}
{'_id': ObjectId('5c719e0a3d10166478cc358f'),
 'name': "Naked Fish's Sushi & Grill"}
{'_id': ObjectId('5c719e0a3d10166478cc4e71'), 'name': 'Tonkatsu Kiyoshi'}
{'_id': ObjectId('5c719e0a3d10166478cc8e3e'), 'name': 'Shang Artisan Noodle'}
{'_id': ObjectId('5c719e0a3d10166478cc8ea4'), 'name': 'Jjanga Steak & Sushi'}
{'_id': ObjectId('5c719e0a3d10166478cc94d6'),
 'name': 'Soho Japanese Restaurant'}
{'_id': ObjectId('5c719e0a3d10166478cca5ff'), 'name': 'Sweets Raku'}


In [5]:
# Count of review per city (Aggregate), sorted by counts
myaggregate = [{
    "$match": {'is_open':1}}, 
    {"$group": {'_id': '$city',
               'total': {'$sum': '$review_count'}}},
    {'$sort': {'total':-1}}]

aggregate_result = mydb['business'].aggregate(myaggregate)
for record in aggregate_result:
    print(record)

{'total': 5100993, '_id': 'Las Vegas'}
{'total': 1865718, '_id': 'Phoenix'}
{'total': 1267338, '_id': 'Toronto'}
{'total': 951900, '_id': 'Scottsdale'}
{'total': 795114, '_id': 'Charlotte'}
{'total': 574827, '_id': 'Pittsburgh'}
{'total': 556995, '_id': 'Henderson'}
{'total': 506871, '_id': 'Tempe'}
{'total': 435267, '_id': 'Mesa'}
{'total': 394278, '_id': 'Montréal'}
{'total': 391779, '_id': 'Chandler'}
{'total': 335211, '_id': 'Gilbert'}
{'total': 293106, '_id': 'Cleveland'}
{'total': 262719, '_id': 'Madison'}
{'total': 258030, '_id': 'Glendale'}
{'total': 245271, '_id': 'Calgary'}
{'total': 148860, '_id': 'Peoria'}
{'total': 142626, '_id': 'Mississauga'}
{'total': 131478, '_id': 'North Las Vegas'}
{'total': 127275, '_id': 'Markham'}
{'total': 88980, '_id': 'Surprise'}
{'total': 76803, '_id': 'Goodyear'}
{'total': 75648, '_id': 'Champaign'}
{'total': 59631, '_id': 'Richmond Hill'}
{'total': 59580, '_id': 'Avondale'}
{'total': 50505, '_id': 'North York'}
{'total': 48915, '_id': 'Conco

Join name of entities to tip to find out the name of the place receiving the most tips

In [6]:
# aggregate tips
group_query = [{
    '$group'  :{
        '_id': '$business_id',
        'compliment_total': {'$sum': '$compliment_count'}
    }},{
    '$out': 'tip_aggregate_by_business'}]

group_result = mydb['tip'].aggregate(group_query)

In [7]:
# select name and business_id from business
group_query = [{
    '$project':{
        'business_id':1,
        'name':1
    }},
    {
        '$group': {
            '_id': '$business_id',
            'business_name': {'$first':'$name'}
        }
    },
    {'$out': 'business_name'}]

group_result = mydb['business'].aggregate(group_query)

In [8]:
# join and sort
join_query = [
{
    '$lookup':{
        'from': 'business_name',
        'localField': '_id',
        'foreignField': '_id',
        'as': 'business_info'
    }}
    ,{
    '$project': {
        'compliment_total':1,
        'business_info.business_name':1,
        '_id':1
    }}
    ,
    {
    '$sort':{'compliment_total':-1}
    },
    {
    '$limit':10
    }
]

join_result = mydb['tip_aggregate_by_business'].aggregate(join_query)

In [9]:
for res in join_result:
    pprint.pprint(res)

{'_id': 'BQqwIYQuo2W94smjrBjy5g',
 'business_info': [{'business_name': 'Il Chianti Italian Steak & Seafood'}],
 'compliment_total': 204}
{'_id': 'FaHADZARwnY4yvlvpnsfGA',
 'business_info': [{'business_name': 'McCarran International Airport'}],
 'compliment_total': 162}
{'_id': 'RESDUcs7fIiihp38-d6_6g',
 'business_info': [{'business_name': 'Bacchanal Buffet'}],
 'compliment_total': 150}
{'_id': '55E0-qUHa7Kzqz8rOhbdBQ',
 'business_info': [{'business_name': 'Costco Gasoline'}],
 'compliment_total': 120}
{'_id': 'MpmFFw0GE_2iRFPdsRpJbA',
 'business_info': [{'business_name': 'XS Nightclub'}],
 'compliment_total': 111}
{'_id': 'JmI9nslLD7KZqRr__Bg6NQ',
 'business_info': [{'business_name': 'Phoenix Sky Harbor International '
                                     'Airport'}],
 'compliment_total': 105}
{'_id': 'DN0b4Un8--Uf6SEWLeh0UA',
 'business_info': [{'business_name': "Amy's Baking Company"}],
 'compliment_total': 102}
{'_id': 'QsKhwKYB3YeWXqpIPd5QMg',
 'business_info': [{'business_name': '